# Mola implementation

In [1]:
import mola.build as mb
from pyomo.environ import *
from pyomo.environ import units as pu
config = mb.get_config('../config/Lemon_Toy_Model.json')
model = mb.build_instance(config)
opt = SolverFactory("glpk")
model.obj.deactivate()
model.obj2.deactivate()
model.obj1.activate()
results = opt.solve(model)

In [2]:
d_units = pu.get_units(model.d['64867712-23c4-3be5-a50e-3631e74571a6', '9891d347-28f8-434d-896a-84a19a4c68a6', 'k1', 't1'])
d_units.pprint()

km

In [3]:
t = model.Specific_Transport_Flow
t.pprint()

Specific_Transport_Flow : Specific Transport Flow
    Size=9, Index=Specific_Transport_Flow_index
    Key                                                                                          : Lower : Value            : Upper : Fixed : Stale : Domain
    ('090f56b8-6a83-48c1-ae26-234d04771e53', '13e79207-71e6-36c9-be76-716700ea1e81', 'k1', 't1') :     0 :              0.0 :  None : False : False : NonNegativeReals
    ('090f56b8-6a83-48c1-ae26-234d04771e53', '3886c7a5-bb83-41d8-9b20-41c0df9742ce', 'k1', 't1') :     0 :              0.0 :  None : False : False : NonNegativeReals
    ('090f56b8-6a83-48c1-ae26-234d04771e53', 'b0d5d6ec-77f4-3213-b613-2391089d5f96', 'k1', 't1') :     0 :              0.0 :  None : False : False : NonNegativeReals
    ('0ace02fa-eca5-482d-a829-c18e46a52db4', '13e79207-71e6-36c9-be76-716700ea1e81', 'k1', 't1') :     0 :              0.0 :  None : False : False : NonNegativeReals
    ('0ace02fa-eca5-482d-a829-c18e46a52db4', '3886c7a5-bb83-41d8-9b20-41c0df9

Units of transport flow depend on the transport process.

In [4]:
pu.get_units(t).pprint()

P_t

In [5]:
t0 = t['090f56b8-6a83-48c1-ae26-234d04771e53', '13e79207-71e6-36c9-be76-716700ea1e81', 'k1', 't1']
t0.pprint()

{Member of Specific_Transport_Flow} : Specific Transport Flow
    Size=9, Index=Specific_Transport_Flow_index
    Key                                                                                          : Lower : Value : Upper : Fixed : Stale : Domain
    ('090f56b8-6a83-48c1-ae26-234d04771e53', '13e79207-71e6-36c9-be76-716700ea1e81', 'k1', 't1') :     0 :   0.0 :  None : False : False : NonNegativeReals


In [6]:
import mola.dataview as dv
import mola.dataimport as di
conn = di.get_sqlite_connection()
lookup = dv.LookupTables(conn)

This uses the product flow of the process even if the flow in the table does not correspond to that product flow.

In [7]:
import mola.output as mo
mo.get_entity(t, lookup, units=True)

SELECT "TBL_PROCESSES"."REF_ID" "P_t","TBL_UNITS"."NAME" "Units" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (SELECT "ID" FROM "TBL_PROCESSES" WHERE "REF_ID" IN ('13e79207-71e6-36c9-be76-716700ea1e81','b0d5d6ec-77f4-3213-b613-2391089d5f96','3886c7a5-bb83-41d8-9b20-41c0df9742ce','13e79207-71e6-36c9-be76-716700ea1e81','b0d5d6ec-77f4-3213-b613-2391089d5f96','3886c7a5-bb83-41d8-9b20-41c0df9742ce','13e79207-71e6-36c9-be76-716700ea1e81','b0d5d6ec-77f4-3213-b613-2391089d5f96','3886c7a5-bb83-41d8-9b20-41c0df9742ce'))) "sq0" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="sq0"."F_FLOW" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="sq0"."F_OWNER" LEFT JOIN "TBL_UNITS" ON "TBL_UNITS"."ID"="sq0"."F_UNIT" WHERE "TBL_FLOWS"."FlOW_TYPE"='PRODUCT_FLOW'


,Specific_Transport_Flow,Units,F_t,P_t
0,0.000000,t*km,"transport, freight, light commercial vehicle","transport, freight, light commercial vehicle |..."
1,0.000000,t*km,"transport, freight, light commercial vehicle","transport, freight train, diesel | transport, ..."
2,0.000000,t*km,"transport, freight, light commercial vehicle","transport, freight, sea, container ship | tran..."
3,0.000000,t*km,"transport, freight train","transport, freight, light commercial vehicle |..."
4,0.000000,t*km,"transport, freight train","transport, freight train, diesel | transport, ..."
5,0.000000,t*km,"transport, freight train","transport, freight, sea, container ship | tran..."
6,0.000000,t*km,"transport, freight, sea, container ship","transport, freight, light commercial vehicle |..."
7,0.000000,t*km,"transport, freight, sea, container ship","transport, freight train, diesel | transport, ..."
8,1283.834155,t*km,"transport, freight, sea, container ship","transport, freight, sea, container ship | tran..."


Current explicit unit conversion factor from kg km to t km.

In [8]:
model.U.pprint()

U : Conversion factor for material flow units in transport flow units
    Size=3, Index=U_index, Domain=Reals, Default=None, Mutable=True
    Key                                                                              : Value
    ('9891d347-28f8-434d-896a-84a19a4c68a6', '090f56b8-6a83-48c1-ae26-234d04771e53') : 0.001
    ('9891d347-28f8-434d-896a-84a19a4c68a6', '0ace02fa-eca5-482d-a829-c18e46a52db4') : 0.001
    ('9891d347-28f8-434d-896a-84a19a4c68a6', '2741cea8-327f-4e0f-9401-b10858dc68f8') : 0.001


Can we get the conversion factor implicitly using pyomo unit conversion?

In [9]:
P_t = list(model.P_t)
dv.get_process_product_flow_units(conn, P_t)

SELECT "TBL_PROCESSES"."REF_ID" "P","TBL_UNITS"."NAME" "Units" FROM (SELECT "F_OWNER","F_FLOW","F_UNIT","RESULTING_AMOUNT_VALUE" FROM "TBL_EXCHANGES" WHERE "F_OWNER" IN (SELECT "ID" FROM "TBL_PROCESSES" WHERE "REF_ID" IN ('13e79207-71e6-36c9-be76-716700ea1e81','b0d5d6ec-77f4-3213-b613-2391089d5f96','3886c7a5-bb83-41d8-9b20-41c0df9742ce'))) "sq0" LEFT JOIN "TBL_FLOWS" ON "TBL_FLOWS"."ID"="sq0"."F_FLOW" LEFT JOIN "TBL_PROCESSES" ON "TBL_PROCESSES"."ID"="sq0"."F_OWNER" LEFT JOIN "TBL_UNITS" ON "TBL_UNITS"."ID"="sq0"."F_UNIT" WHERE "TBL_FLOWS"."FlOW_TYPE"='PRODUCT_FLOW'


,Units
P,
b0d5d6ec-77f4-3213-b613-2391089d5f96,t*km
3886c7a5-bb83-41d8-9b20-41c0df9742ce,t*km
13e79207-71e6-36c9-be76-716700ea1e81,t*km


In [10]:
unit_map = mb.map_reference_units()
unit_map

AttributeError: module 'mola.build' has no attribute 'map_reference_units'

If we specifiy the specific transport flow with units kg km then this does not correspond to the units of the transport flow in openLCA. In the transport constraint we need to multiply the transport expression inside the summation by

In [ ]:
pu.convert_value(1, from_units=unit_map['kg']*pu.km, to_units=unit_map['t*km'])

# DataPortal and Units

In [ ]:
model = AbstractModel()
data = DataPortal()
model.A = Set(initialize=['A1','A2','A3'])
model.y = Param(model.A)
data.load(filename='Y.tab', param=model.y)
instance = model.create_instance(data)

In [ ]:
instance.y.pprint()

In [ ]:
instance.p = Param(instance.A, initialize={'A1':'kg', 'A2':'tonne km', 'A3': 'tonne'}, within=Character)
instance.p.pprint()

In [ ]:
type(instance.p['A1'])